In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import os

folder_path = "/content/drive/My Drive/CICIDS 2017/"  # Update with correct path

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read and combine all CSV files
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]

# Concatenate all DataFrames into one
df = pd.concat(df_list, ignore_index=True)

print(df.head())  # Display first few rows


    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              55055              52                   1   
3              46236              34                   1   
4              54863               3                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        0                           12   
1                        1                            6   
2                        1                            6   
3                        1                            6   
4                        0                           12   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                             6                       6   
2                             6                       6   
3                             6                 

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import pickle
from os import path

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/CICIDS 2017/"

Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Friday-WorkingHours-Morning.pcap_ISCX.csv
Monday-WorkingHours.pcap_ISCX.csv
Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Tuesday-WorkingHours.pcap_ISCX.csv
Wednesday-workingHours.pcap_ISCX.csv


In [ ]:
data = pd.read_csv('/content/drive/My Drive/CICIDS 2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# --- **IMPORTANT:** Replace with the actual path to your CSV file ---
file_path = '/content/drive/My Drive/CICIDS 2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
# Example: file_path = '/content/drive/My Drive/CICIDS 2017/Monday-workingHours.pcap_ISCX.csv'

try:
    data = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
    print("\nFirst 5 rows of the dataset:")
    print(data.head())
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the file path.")
    exit()

# Select numerical features
numerical_features = data.select_dtypes(include=np.number).columns

print("\nIdentified numerical features:", numerical_features.tolist())

# --- **Handle infinite and very large values** ---
print("\nChecking for infinite values in numerical columns...")
if len(numerical_features) > 0:
    infinite_cols = data[numerical_features].columns[np.isinf(data[numerical_features]).any()].tolist()
    print("Columns with infinite values:", infinite_cols)
    data[numerical_features].replace([np.inf, -np.inf], np.nan, inplace=True)
    print("Infinite values in numerical columns replaced with NaN.")

    large_threshold = 1e15  # Adjust as needed
    print(f"\nChecking for very large values (>{large_threshold}) in numerical columns...")
    for col in numerical_features:
        if pd.api.types.is_numeric_dtype(data[col]):  # Ensure column is numeric before comparison
            if (data[col].astype(float) > large_threshold).any():
                print(f"Column '{col}' has very large values.")
                data[col].where(data[col].astype(float) <= large_threshold, np.nan, inplace=True)
    print("Very large values in numerical columns replaced with NaN.")

    # --- **Handle NaN values (imputation)** ---
    print("\nHandling NaN values in numerical columns (filling with mean)...")
    data[numerical_features].fillna(data[numerical_features].mean(), inplace=True)
    print("NaN values in numerical columns filled with the mean.")

    # Create copies for scaled data
    data_minmax = data.copy()
    data_standard = data.copy()

    # Normalize using Min-Max scaling on numerical features
    min_max_scaler = MinMaxScaler()
    data_minmax[numerical_features] = min_max_scaler.fit_transform(data_minmax[numerical_features])

    print("\nFirst 5 rows after Min-Max Normalization:")
    print(data_minmax.head())

    # Standardize using Z-score scaling on numerical features
    standard_scaler = StandardScaler()
    data_standard[numerical_features] = standard_scaler.fit_transform(data_standard[numerical_features])

    print("\nFirst 5 rows after Standardization (Z-score):")
    print(data_standard.head())

else:
    print("\nNo numerical features found in the dataset. Skipping normalization and standardization.")

print("\nData pre-processing using Min-Max Normalization and Standardization (Z-score) complete (if numerical features were found).")

# --- Important Notes: ---
# 1. **File Path:** Ensure the `file_path` variable at the beginning of the script correctly points to your CSV file.
# 2. **Numerical Features:** The code automatically identifies numerical features. Review the output of "Identified numerical features" to ensure it's correct.
# 3. **Handling Infinite and Large Values:**
#    - Infinite values (inf, -inf) are replaced with NaN.
#    - Very large values (exceeding a threshold) are also replaced with NaN. Adjust `large_threshold` if needed.
# 4. **Missing Values:** NaN values in numerical columns are filled with the mean of each column. Consider other imputation methods if appropriate for your data.
# 5. **Scaling:** Min-Max Normalization scales features to the range [0, 1]. Standardization (Z-score) scales features to have a mean of 0 and a standard deviation of 1.
# 6. **Separate Scaled DataFrames:** The normalized data is stored in `data_minmax`, and the standardized data is in `data_standard`. The original `data` DataFrame is also preserved (after handling infinite/large/NaN values).
# 7. **No Numerical Features:** The code now handles the case where no numerical features are found in the dataset.

Dataset loaded successfully.

First 5 rows of the dataset:
    Destination Port   Flow Duration   Total Fwd Packets  \
0                389       113095465                  48   
1                389       113473706                  68   
2                  0       119945515                 150   
3                443        60261928                   9   
4                 53             269                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                       24                         9668   
1                       40                        11364   
2                        0                            0   
3                        7                         2330   
4                        2                          102   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                         10012                     403   
1                         12718                     403   
2                             0                 

<ipython-input-9-8317bfc14c8f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numerical_features].replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-9-8317bfc14c8f>:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].where(data[col].astype(float) <= large_threshold, np.nan, inplace=True)
<ipython-input-9-8317bfc14c

NaN values in numerical columns filled with the mean.

First 5 rows after Min-Max Normalization:
    Destination Port   Flow Duration   Total Fwd Packets  \
0           0.005936        0.942462            0.000234   
1           0.005936        0.945614            0.000334   
2           0.000000        0.999546            0.000742   
3           0.006760        0.502183            0.000040   
4           0.000809        0.000002            0.000005   

    Total Backward Packets  Total Length of Fwd Packets  \
0                 0.000089                     0.008076   
1                 0.000148                     0.009492   
2                 0.000000                     0.000000   
3                 0.000026                     0.001946   
4                 0.000007                     0.000085   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                  1.596810e-05                0.017252   
1                  2.028389e-05                0.017252   
2         

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# --- **IMPORTANT:** Replace with the actual path to your CSV file ---
file_path ='/content/drive/My Drive/CICIDS 2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
# Example: file_path = '/content/drive/My Drive/CICIDS 2017/Monday-workingHours.pcap_ISCX.csv'

# --- Load and check data in a separate cell ---
try:
    data = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
    print("\nFirst 5 rows of the dataset:")
    print(data.head())
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the file path.")
    exit()

# --- Start a new cell for the remaining code ---

# Identify the target variable column name
target_column = 'Label'  # **IMPORTANT:** Replace 'Label' with the actual name of your target variable column

# Select numerical features for normalization and standardization
# --- Check if the target column exists in the DataFrame ---
if target_column in data.columns:
    numerical_features = data.select_dtypes(include=np.number).columns
else:
    print(f"Error: Target column '{target_column}' not found in the dataset.")
    exit()

# --- Now continue with the rest of your feature selection and preprocessing code ---

print("\nIdentified numerical features:", numerical_features.tolist())
# --- **Handle infinite and very large values** ---
print("\nChecking for infinite values in numerical columns...")
if len(numerical_features) > 0:
    infinite_cols = data[numerical_features].columns[np.isinf(data[numerical_features]).any()].tolist()
    print("Columns with infinite values:", infinite_cols)
    data[numerical_features].replace([np.inf, -np.inf], np.nan, inplace=True)
    print("Infinite values in numerical columns replaced with NaN.")

    # ... (Rest of your code remains the same)

    # ... (Rest of your code remains the same)

    large_threshold = 1e15  # Adjust as needed
    print(f"\nChecking for very large values (>{large_threshold}) in numerical columns...")
    for col in numerical_features:
        if pd.api.types.is_numeric_dtype(data[col]):  # Ensure column is numeric before comparison
            if (data[col].astype(float) > large_threshold).any():
                print(f"Column '{col}' has very large values.")
                data[col].where(data[col].astype(float) <= large_threshold, np.nan, inplace=True)
    print("Very large values in numerical columns replaced with NaN.")

    # --- **Handle NaN values (imputation)** ---
    print("\nHandling NaN values in numerical columns (filling with mean)...")
    data[numerical_features].fillna(data[numerical_features].mean(), inplace=True)
    print("NaN values in numerical columns filled with the mean.")

    # Create copies for scaled data
    data_standard = data.copy()

    # Standardize using Z-score scaling on numerical features
    standard_scaler = StandardScaler()
    data_standard[numerical_features] = standard_scaler.fit_transform(data_standard[numerical_features])

    print("\nFirst 5 rows after Standardization (Z-score):")
    print(data_standard.head())

    # --- Feature Selection ---
    # Check if the target column exists in the DataFrame
    if target_column not in data_standard.columns:
        print(f"\nError: Target variable column '{target_column}' not found in the preprocessed data.")
    else:
        # Separate features (X) and target (y)
        X = data_standard.drop(target_column, axis=1)
        y = data_standard[target_column]

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Decision Tree
        dt_classifier = DecisionTreeClassifier(random_state=42)
        dt_classifier.fit(X_train, y_train)
        dt_y_pred = dt_classifier.predict(X_test)
        dt_accuracy = accuracy_score(y_test, dt_y_pred)
        print(f"\nDecision Tree Accuracy: {dt_accuracy}")

        # Feature importances from Decision Tree
        dt_feature_importances = dt_classifier.feature_importances_
        dt_important_features = pd.Series(dt_feature_importances, index=X.columns).sort_values(ascending=False)
        print("\nDecision Tree Feature Importances:")
        print(dt_important_features)

        # Random Forest
        rf_classifier = RandomForestClassifier(random_state=42)
        rf_classifier.fit(X_train, y_train)
        rf_y_pred = rf_classifier.predict(X_test)
        rf_accuracy = accuracy_score(y_test, rf_y_pred)
        print(f"\nRandom Forest Accuracy: {rf_accuracy}")

        # Feature importances from Random Forest
        rf_feature_importances = rf_classifier.feature_importances_
        rf_important_features = pd.Series(rf_feature_importances, index=X.columns).sort_values(ascending=False)
        print("\nRandom Forest Feature Importances:")
        print(rf_important_features)

        # --- CNN Feature Selection (Illustrative Example) ---
        # CNNs are not directly used for feature selection in the same way as trees.
        # They learn features during training. You can examine the learned filters or
        # weights to understand feature importance but it's a more complex process.
        # Here is an illustrative example using the output of the first layer of a simple CNN
        # which might represent the extracted features:

        # Check if the number of features is greater than 0 before proceeding with CNN
        if X_train.shape[1] > 0:
            # Example (requires conversion of data into suitable format for CNN)
            # Assume data reshaped into num_samples, timesteps, features for CNN
            # You might need to reshape or otherwise process your data
            X_cnn = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1]) # Example reshape

            # Determine the number of unique classes in the target variable for the output layer
            num_classes = len(np.unique(y_train))

            model = keras.Sequential([
                Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1, X_train.shape[1])),
                MaxPooling1D(pool_size=2),
                Flatten(),
                Dense(num_classes, activation='softmax') # Output layer based on the number of classes
            ])
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            model.fit(X_cnn, y_train, epochs=10, verbose=0) # Train for limited epochs for illustration

            # Extract first layer feature maps
            intermediate_layer_model = keras.Model(inputs=model.input, outputs=model.layers[0].output)
            intermediate_output = intermediate_layer_model.predict(X_cnn)

            print(f"\nShape of CNN's first layer output: {intermediate_output.shape}")
            print("\nCNN: Examining the learned filters or weights of the first layer can provide insights into which features the CNN finds important. This requires further analysis of the `model.layers[0].get_weights()`.")
        else:
            print("\nCNN: No features available for CNN model.")

else:
    print("\nNo numerical features found in the dataset. Skipping feature selection.")

print("\nFeature selection using Decision Tree, Random Forest, and CNN (illustrative) complete (if numerical features and target variable were found).")

Dataset loaded successfully.

First 5 rows of the dataset:
    Destination Port   Flow Duration   Total Fwd Packets  \
0                389       113095465                  48   
1                389       113473706                  68   
2                  0       119945515                 150   
3                443        60261928                   9   
4                 53             269                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                       24                         9668   
1                       40                        11364   
2                        0                            0   
3                        7                         2330   
4                        2                          102   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                         10012                     403   
1                         12718                     403   
2                             0                 

<ipython-input-10-fe97102144ae>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numerical_features].replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-10-fe97102144ae>:60: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].where(data[col].astype(float) <= large_threshold, np.nan, inplace=True)
<ipython-input-10-fe97102


First 5 rows after Standardization (Z-score):
    Destination Port   Flow Duration   Total Fwd Packets  \
0          -0.411731        3.150810            0.029272   
1          -0.411731        3.162653            0.047080   
2          -0.433063        3.365287            0.120092   
3          -0.408770        1.496579           -0.005453   
4          -0.430156       -0.390228           -0.011686   

    Total Backward Packets  Total Length of Fwd Packets  \
0                 0.004000                     1.181651   
1                 0.014706                     1.401614   
2                -0.012059                    -0.072238   
3                -0.007375                     0.229951   
4                -0.010721                    -0.059009   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                     -0.006305                0.509918   
1                     -0.005523                0.509918   
2                     -0.009198               -0.363704   
3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape

# --- **IMPORTANT:** Replace with the actual path to your CSV file ---
file_path = '/content/drive/My Drive/CICIDS 2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
# Example: file_path = '/content/drive/My Drive/CICIDS 2017/Monday-workingHours.pcap_ISCX.csv'

# --- **SOLUTION:** Correct the target variable column name ---
# The traceback indicates a KeyError: "['Label'] not found in axis".
# The error message printed previously also shows the available columns.
# Based on the printed list of columns, the target variable column name is ' Label' (note the leading space).
# Replace 'Label' below with the actual target column name including the space.
target_column = ' Label'  # **REPLACE 'Label' WITH THE ACTUAL TARGET COLUMN NAME (including space)**

try:
    data = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
    print("\nFirst 5 rows of the dataset:")
    print(data.head())

    # Separate features (X) and target (y)
    # --- **SOLUTION:** Check if the target column exists BEFORE trying to drop it ---
    if target_column not in data.columns:
        print(f"\nError: Target variable column '{target_column}' not found in the dataset columns:")
        print(data.columns.tolist())  # Print the actual column names for debugging
        print("Please update the 'target_column' variable with the correct name (including any leading/trailing spaces).")
        exit()
    X = data.drop(target_column, axis=1)
    y = data[target_column]

    # Identify numerical and categorical features
    numerical_features = X.select_dtypes(include=np.number).columns
    categorical_features = X.select_dtypes(include=['object', 'category']).columns

    # --- Preprocessing ---
    # Handle missing values (impute numerical with mean, categorical with mode)
    for col in numerical_features:
        X[col].fillna(X[col].mean(), inplace=True)
    for col in categorical_features:
        if not X[col].empty:  # Check if the column is not empty before taking mode
            X[col].fillna(X[col].mode()[0], inplace=True)
        else:
            print(f"Warning: Categorical column '{col}' is empty and cannot be imputed with mode.")

    # Encode categorical features
    X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Encode the target variable if it's not already numerical
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    num_classes = len(np.unique(y_train_encoded))
    print(f"\nNumber of classes: {num_classes}")

    # --- **SOLUTION:** Handle potential infinite or very large values before scaling ---
    print("\nChecking for infinite or very large values in numerical features...")
    numerical_cols_x_train = X_train.select_dtypes(include=np.number).columns
    numerical_cols_x_test = X_test.select_dtypes(include=np.number).columns

    for col in numerical_cols_x_train:
        X_train[col] = np.where(np.isinf(X_train[col]), np.nan, X_train[col])
        X_train[col] = np.where(np.abs(X_train[col]) > 1e10, np.nan, X_train[col])  # Adjust threshold as needed
    for col in numerical_cols_x_test:
        X_test[col] = np.where(np.isinf(X_test[col]), np.nan, X_test[col])
        X_test[col] = np.where(np.abs(X_test[col]) > 1e10, np.nan, X_test[col])  # Adjust threshold as needed

    # Impute any remaining NaNs after handling inf/large values
    for col in numerical_cols_x_train:
        X_train[col].fillna(X_train[col].mean(), inplace=True)
    for col in numerical_cols_x_test:
        X_test[col].fillna(X_test[col].mean(), inplace=True)

    # Scale numerical features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert scaled data back to DataFrame for easier handling (optional)
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    # --- Multiclassification Models ---

    print("\n--- Decision Tree ---")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_scaled_df, y_train_encoded)
    dt_y_pred = dt_classifier.predict(X_test_scaled_df)
    dt_accuracy = accuracy_score(y_test_encoded, dt_y_pred)
    print(f"Accuracy: {dt_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, dt_y_pred))

    print("\n--- Random Forest ---")
    rf_classifier = RandomForestClassifier(random_state=42)
    rf_classifier.fit(X_train_scaled_df, y_train_encoded)
    rf_y_pred = rf_classifier.predict(X_test_scaled_df)
    rf_accuracy = accuracy_score(y_test_encoded, rf_y_pred)
    print(f"Accuracy: {rf_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, rf_y_pred))

    print("\n--- Logistic Regression ---")
    lr_classifier = LogisticRegression(random_state=42, solver='liblinear', multi_class='ovr', max_iter=1000)
    lr_classifier.fit(X_train_scaled_df, y_train_encoded)
    lr_y_pred = lr_classifier.predict(X_test_scaled_df)
    lr_accuracy = accuracy_score(y_test_encoded, lr_y_pred)
    print(f"Accuracy: {lr_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, lr_y_pred))

    print("\n--- Support Vector Machine (SVM) ---")
    svm_classifier = SVC(random_state=42, kernel='rbf', C=1.0)
    svm_classifier.fit(X_train_scaled_df, y_train_encoded)
    svm_y_pred = svm_classifier.predict(X_test_scaled_df)
    svm_accuracy = accuracy_score(y_test_encoded, svm_y_pred)
    print(f"Accuracy: {svm_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, svm_y_pred))

    print("\n--- Gaussian Naive Bayes ---")
    gnb_classifier = GaussianNB()
    gnb_classifier.fit(X_train_scaled_df, y_train_encoded)
    gnb_y_pred = gnb_classifier.predict(X_test_scaled_df)
    gnb_accuracy = accuracy_score(y_test_encoded, gnb_y_pred)
    print(f"Accuracy: {gnb_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, gnb_y_pred))

    print("\n--- Convolutional Neural Network (CNN) ---")
    # Reshape input data for CNN (assuming 1 time step)
    X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

    cnn_model = keras.Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    cnn_model.summary()

    # Train the CNN
    epochs = 10
    batch_size = 32
    cnn_model.fit(X_train_cnn, y_train_encoded, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the CNN
    cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test_encoded, verbose=0)
    print(f"CNN Accuracy: {cnn_accuracy:.4f}")

    print("\nMulticlassification using various techniques complete.")

except FileNotFoundError:
    print(f"\nError: The file '{file_path}' was not found. Please check the file path.")
    exit()

Dataset loaded successfully.

First 5 rows of the dataset:
    Destination Port   Flow Duration   Total Fwd Packets  \
0                389       113095465                  48   
1                389       113473706                  68   
2                  0       119945515                 150   
3                443        60261928                   9   
4                 53             269                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                       24                         9668   
1                       40                        11364   
2                        0                            0   
3                        7                         2330   
4                        2                          102   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                         10012                     403   
1                         12718                     403   
2                             0                 

<ipython-input-4-c3c9e3ce40fa>:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].mean(), inplace=True)



Number of classes: 4

Checking for infinite or very large values in numerical features...


<ipython-input-4-c3c9e3ce40fa>:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(X_train[col].mean(), inplace=True)
<ipython-input-4-c3c9e3ce40fa>:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr


--- Decision Tree ---
Accuracy: 0.9954
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     33638
           1       0.75      0.74      0.74       302
           2       0.50      0.75      0.60         4
           3       0.42      0.42      0.42       130

    accuracy                           1.00     34074
   macro avg       0.67      0.73      0.69     34074
weighted avg       1.00      1.00      1.00     34074


--- Random Forest ---
Accuracy: 0.9953
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     33638
           1       0.74      0.77      0.75       302
           2       0.67      0.50      0.57         4
           3       0.40      0.33      0.36       130

    accuracy                           1.00     34074
   macro avg       0.70      0.65      0.67     34074
weighted avg       1.00      1.00      1.00     34074


-

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 76, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 38, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 36, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 18, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,516 (1.22 MB)

 Trainable params: 320,516 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, make_scorer, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape

# --- **IMPORTANT:** Replace with the actual path to your CSV file ---
file_path = '/content/drive/My Drive/CICIDS 2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
# Example: file_path = '/content/drive/My Drive/CICIDS 2017/Monday-workingHours.pcap_ISCX.csv'

# --- **SOLUTION:** Correct the target variable column name ---
target_column = ' Label'  # **REPLACE 'Label' WITH THE ACTUAL TARGET COLUMN NAME (including space)**

try:
    data = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
    print("\nFirst 5 rows of the dataset:")
    print(data.head())

    # Separate features (X) and target (y)
    if target_column not in data.columns:
        print(f"\nError: Target variable column '{target_column}' not found in the dataset columns:")
        print(data.columns.tolist())
        print("Please update the 'target_column' variable with the correct name (including any leading/trailing spaces).")
        exit()
    X = data.drop(target_column, axis=1)
    y = data[target_column]

    # Identify numerical and categorical features
    numerical_features = X.select_dtypes(include=np.number).columns
    categorical_features = X.select_dtypes(include=['object', 'category']).columns

    # --- Preprocessing ---
    # Handle missing values (impute numerical with mean, categorical with mode)
    for col in numerical_features:
        X[col].fillna(X[col].mean(), inplace=True)
    for col in categorical_features:
        if not X[col].empty:
            X[col].fillna(X[col].mode()[0], inplace=True)
        else:
            print(f"Warning: Categorical column '{col}' is empty and cannot be imputed with mode.")

    # Encode categorical features
    X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

    # Split data into training and testing sets with stratification
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Encode the target variable if it's not already numerical
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    num_classes = len(np.unique(y_train_encoded))
    print(f"\nNumber of classes: {num_classes}")

    # --- Handle potential infinite or very large values before scaling ---
    print("\nChecking for infinite or very large values in numerical features...")
    numerical_cols_x_train = X_train.select_dtypes(include=np.number).columns
    numerical_cols_x_test = X_test.select_dtypes(include=np.number).columns

    for col in numerical_cols_x_train:
        X_train[col] = np.where(np.isinf(X_train[col]), np.nan, X_train[col])
        X_train[col] = np.where(np.abs(X_train[col]) > 1e10, np.nan, X_train[col])  # Adjust threshold as needed
    for col in numerical_cols_x_test:
        X_test[col] = np.where(np.isinf(X_test[col]), np.nan, X_test[col])
        X_test[col] = np.where(np.abs(X_test[col]) > 1e10, np.nan, X_test[col])  # Adjust threshold as needed

    # Impute any remaining NaNs after handling inf/large values
    for col in numerical_cols_x_train:
        X_train[col].fillna(X_train[col].mean(), inplace=True)
    for col in numerical_cols_x_test:
        X_test[col].fillna(X_test[col].mean(), inplace=True)

    # Scale numerical features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert scaled data back to DataFrame for easier handling (optional)
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    # --- Multiclassification Models ---

    print("\n--- Decision Tree ---")
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train_scaled_df, y_train_encoded)
    dt_y_pred = dt_classifier.predict(X_test_scaled_df)
    dt_accuracy = accuracy_score(y_test_encoded, dt_y_pred)
    print(f"Accuracy: {dt_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, dt_y_pred))

    print("\n--- Random Forest ---")
    rf_classifier = RandomForestClassifier(random_state=42)
    rf_classifier.fit(X_train_scaled_df, y_train_encoded)
    rf_y_pred = rf_classifier.predict(X_test_scaled_df)
    rf_accuracy = accuracy_score(y_test_encoded, rf_y_pred)
    print(f"Accuracy: {rf_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, rf_y_pred))

    print("\n--- Logistic Regression ---")
    lr_classifier = LogisticRegression(random_state=42, solver='liblinear', multi_class='ovr', max_iter=1000)
    if num_classes >= 2:
        lr_classifier.fit(X_train_scaled_df, y_train_encoded)
        lr_y_pred = lr_classifier.predict(X_test_scaled_df)
        lr_accuracy = accuracy_score(y_test_encoded, lr_y_pred)
        print(f"Accuracy: {lr_accuracy:.4f}")
        print("Classification Report:\n", classification_report(y_test_encoded, lr_y_pred))
    else:
        print("Warning: Logistic Regression requires at least two classes in the training data. Skipping.")

    print("\n--- Support Vector Machine (SVM) ---")
    svm_classifier = SVC(random_state=42, kernel='rbf', C=1.0)
    if num_classes >= 2:
        svm_classifier.fit(X_train_scaled_df, y_train_encoded)
        svm_y_pred = svm_classifier.predict(X_test_scaled_df)
        svm_accuracy = accuracy_score(y_test_encoded, svm_y_pred)
        print(f"Accuracy: {svm_accuracy:.4f}")
        print("Classification Report:\n", classification_report(y_test_encoded, svm_y_pred))
    else:
        print("Warning: SVM requires at least two classes in the training data. Skipping.")

    print("\n--- Gaussian Naive Bayes ---")
    gnb_classifier = GaussianNB()
    gnb_classifier.fit(X_train_scaled_df, y_train_encoded)
    gnb_y_pred = gnb_classifier.predict(X_test_scaled_df)
    gnb_accuracy = accuracy_score(y_test_encoded, gnb_y_pred)
    print(f"Accuracy: {gnb_accuracy:.4f}")
    print("Classification Report:\n", classification_report(y_test_encoded, gnb_y_pred))

    print("\n--- Convolutional Neural Network (CNN) ---")
    # Reshape input data for CNN (assuming 1 time step)
    X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

    cnn_model = keras.Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    cnn_model.summary()

    # Train the CNN
    epochs = 10
    batch_size = 32
    if num_classes >= 2:
        cnn_model.fit(X_train_cnn, y_train_encoded, epochs=epochs, batch_size=batch_size, verbose=0)

        # Evaluate the CNN
        cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test_encoded, verbose=0)
        print(f"CNN Accuracy: {cnn_accuracy:.4f}")
    else:
        print("Warning: CNN requires at least two classes in the training data. Skipping.")

    # --- Performance Estimation using Cross-Validation ---
    print("\n--- Performance Estimation using Cross-Validation ---")

    # Example: Evaluate Decision Tree using 5-fold cross-validation and F1 score
    scoring = make_scorer(f1_score, average='weighted') # Use weighted average for multiclass
    cv_scores_dt = cross_val_score(dt_classifier, X_train_scaled_df, y_train_encoded, cv=5, scoring=scoring)

    print("\nDecision Tree Cross-Validation Scores (F1 Weighted):")
    print(cv_scores_dt)
    print(f"Mean F1 Score: {np.mean(cv_scores_dt):.4f}")
    print(f"Standard Deviation: {np.std(cv_scores_dt):.4f}")

    # Evaluate other models similarly (RandomForest, LogisticRegression, SVM, etc.)

    # Example: RandomForest
    cv_scores_rf = cross_val_score(rf_classifier, X_train_scaled_df, y_train_encoded, cv=5, scoring=scoring)

    print("\nRandom Forest Cross-Validation Scores (F1 Weighted):")
    print(cv_scores_rf)
    print(f"Mean F1 Score: {np.mean(cv_scores_rf):.4f}")
    print(f"Standard Deviation: {np.std(cv_scores_rf):.4f}")

    # Example: Logistic Regression
    if num_classes >= 2:
        cv_scores_lr = cross_val_score(lr_classifier, X_train_scaled_df, y_train_encoded, cv=5, scoring=scoring)
        print("\nLogistic Regression Cross-Validation Scores (F1 Weighted):")
        print(cv_scores_lr)
        print(f"Mean F1 Score: {np.mean(cv_scores_lr):.4f}")
        print(f"Standard Deviation: {np.std(cv_scores_lr):.4f}")
    else:
        print("\nLogistic Regression Cross-Validation Skipped (less than 2 classes).")

    # Example: Support Vector Machine (SVM)
    if num_classes >= 2:
        cv_scores_svm = cross_val_score(svm_classifier, X_train_scaled_df, y_train_encoded, cv=5, scoring=scoring)
        print("\nSupport Vector Machine Cross-Validation Scores (F1 Weighted):")
        print(cv_scores_svm)
        print(f"Mean F1 Score: {np.mean(cv_scores_svm):.4f}")
        print(f"Standard Deviation: {np.std(cv_scores_svm):.4f}")
    else:
        print("\nSupport Vector Machine Cross-Validation Skipped (less than 2 classes).")

    # Example: Gaussian Naive Bayes
    cv_scores_gnb = cross_val_score(gnb_classifier, X_train_scaled_df, y_train_encoded, cv=5, scoring=scoring)
    print("\nGaussian Naive Bayes Cross-Validation Scores (F1 Weighted):")
    print(cv_scores_gnb)
    print(f"Mean F1 Score: {np.mean(cv_scores_gnb):.4f}")
    print(f"Standard Deviation: {np.std(cv_scores_gnb):.4f}")

    print("\nPerformance estimation using cross-validation complete.")

except FileNotFoundError:
    print(f"\nError: The file '{file_path}' was not found. Please check the file path.")
    exit()

Dataset loaded successfully.

First 5 rows of the dataset:
    Destination Port   Flow Duration   Total Fwd Packets  \
0                389       113095465                  48   
1                389       113473706                  68   
2                  0       119945515                 150   
3                443        60261928                   9   
4                 53             269                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                       24                         9668   
1                       40                        11364   
2                        0                            0   
3                        7                         2330   
4                        2                          102   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                         10012                     403   
1                         12718                     403   
2                             0                 

<ipython-input-5-2effb899ecbc>:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].mean(), inplace=True)



Number of classes: 4

Checking for infinite or very large values in numerical features...


<ipython-input-5-2effb899ecbc>:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(X_train[col].mean(), inplace=True)
<ipython-input-5-2effb899ecbc>:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr


--- Decision Tree ---
Accuracy: 0.9954
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     33638
           1       0.75      0.74      0.74       302
           2       0.50      0.75      0.60         4
           3       0.42      0.42      0.42       130

    accuracy                           1.00     34074
   macro avg       0.67      0.73      0.69     34074
weighted avg       1.00      1.00      1.00     34074


--- Random Forest ---
Accuracy: 0.9953
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     33638
           1       0.74      0.77      0.75       302
           2       0.67      0.50      0.57         4
           3       0.40      0.33      0.36       130

    accuracy                           1.00     34074
   macro avg       0.70      0.65      0.67     34074
weighted avg       1.00      1.00      1.00     34074


-

...

 Trainable params: 320,516 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

CNN Accuracy: 0.9932

--- Performance Estimation using Cross-Validation ---

Decision Tree Cross-Validation Scores (F1 Weighted):
[0.99550096 0.9947908  0.99516518 0.99474118 0.99516799]
Mean F1 Score: 0.9951
Standard Deviation: 0.0003

Random Forest Cross-Validation Scores (F1 Weighted):
[0.99563068 0.99527355 0.99485012 0.9950662  0.99533943]
Mean F1 Score: 0.9952
Standard Deviation: 0.0003


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was depre


Logistic Regression Cross-Validation Scores (F1 Weighted):
[0.99049136 0.99036316 0.99078091 0.99063819 0.99119988]
Mean F1 Score: 0.9907
Standard Deviation: 0.0003

Support Vector Machine Cross-Validation Scores (F1 Weighted):
[0.98979796 0.98988538 0.99016226 0.98999711 0.99087129]
Mean F1 Score: 0.9901
Standard Deviation: 0.0004

Gaussian Naive Bayes Cross-Validation Scores (F1 Weighted):
[0.98750952 0.9875257  0.98745726 0.98786009 0.98739902]
Mean F1 Score: 0.9876
Standard Deviation: 0.0002

Performance estimation using cross-validation complete.
